Log in to LinkedIn?



In [ ]:
var notRobot = () => {
    return client.click('')
}

function enterLinkedIn() {
    console.log('LinkedIn: Sign in required');
    var credentials = getCredentials('linkedin.com');
    return client.isExisting('a[href*="uas/login"]')
        .then(is => is ? client.click('a[href*="uas/login"]') : Promise.resolve([]))
        .then(() => client.click('input[name*="session_key"]'))
        .keys(credentials.username)
        .pause(1000)
        .then(() => console.log('LinkedIn: Require password'))
        .click('input[name*="session_password"]')
        .keys(credentials.password)
        .submitForm('.login-form, [type="submit"]')
        .pause(2000)
        .isExisting('.cp-challenge-form')
        .then(is => {
            if (is) {
                throw new Error('captcha');
            }
        });
}

function loginLinkedIn() {
    return client
        .alertText()
        .then(t => t.indexOf('leave') > -1 ? client.alertAccept() : '')
        .catch(e => {
        })
        .getUrl()
        .then(url => {
            var loggedIn = url.indexOf('linkedin') > -1 && url.indexOf('login') == -1 && url.indexOf('authwall') == -1 && url !== 'https://www.linkedin.com';
        console.log(loggedIn);
            return loggedIn
                ? client
                    .pause(1000)
                    .isExisting('iframe.authentication-iframe').then(is => is
                        ? client.element('iframe.authentication-iframe')
                            .then(el => client.frame(el.value))
                            .then(() => enterLinkedIn())
                            .frame()
                        : Promise.resolve([]))
                : client.url('https://www.linkedin.com/')
                    .pause(1000)
                    .isExisting('*=Forgot password?').then(is => is
                        ? enterLinkedIn()
                        : Promise.resolve([]));
        })
};
module.exports = loginLinkedIn;


Scrape LinkedIn profile?



In [ ]:
function visitMyProfile() {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    return client
        .pause(2000)
        .click('#nav-settings__dropdown-trigger')
        .pause(500)
        .click('.nav-settings__view-profile-link')
        .pause(2000);
}

function readLinkedInProfileInfo() {
    var name, title, url, phone, email;
    return client
        .getUrl().then(url => url.indexOf('/in/') == -1
            ? visitMyProfile()
            : Promise.resolve([]))
        .isExisting('.contact-see-more-less')
        .then(is => is ? client.click('.contact-see-more-less') : client)
        .pause(500)
        .then(() => getAllXPath({
            name: '//*[contains(@class, "pv-top-card-section__name")]//text()',
            title: '//*[contains(@class, "pv-top-card-section__headline")]//text()',
            url: '//*[contains(@class, "ci-vanity-url")]//*[contains(@class, "pv-contact-info__contact-item")]//text()',
            phone: '//*[contains(@class, "ci-phone")]//*[contains(@class, "pv-contact-info__contact-item")]//text()',
            email: '//*[contains(@class, "ci-email")]//*[contains(@class, "pv-contact-info__contact-item")]//text()'
        }))
        .then(r => ({
            name: (typeof r.name === 'string' ? [r.name] : (r.name || [])).join('').trim(),
            title: (typeof r.title === 'string' ? [r.title] : (r.title || [])).join('').trim(),
            url: (typeof r.url === 'string' ? [r.url] : (r.url || [])).join('').trim(),
            phone: (typeof r.phone === 'string' ? [r.phone] : (r.phone || [])).join('').trim(),
            email: (typeof r.email === 'string' ? [r.email] : (r.email || [])).join('').trim()
        }));
};
module.exports = readLinkedInProfileInfo;


List LinkedIn threads?



In [ ]:
function scrollLinkedInThreads(threads) {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .execute(() => {
            document.getElementsByClassName(
                'msg-conversations-container__conversations-list')[0]
                .scrollTop += 10000;
        })
        .pause(2000)
        .then(() => getLinkedInThreads(threads));
}

function getLinkedInThreads(threads) {
    return client
        .execute(() => {
            var iterator = document.evaluate(
                '//*[@data-control-name="view_message"]/@href',
                document, null,
                XPathResult.UNORDERED_NODE_ITERATOR_TYPE, null);
            var co = [];
            var m;
            while ((m = iterator.iterateNext()) && co.push(m.value)) ;
            return co;
        })
        .then(els => {
            var newThreads = els.value.filter(e => threads.indexOf(e) === -1);
            threads = threads.concat(newThreads);
            return newThreads.length > 0
                ? scrollLinkedInThreads(threads)
                : Promise.resolve(threads);
        });
}

function listLinkedInThreads(threads) {
    return client.getUrl().then(url => url.indexOf('/messaging') == -1
        ? client.click('a[href*="/messaging/"]')
        : Promise.resolve([]))
        .pause(2000)
        .then(() => getLinkedInThreads(threads || []))
};
module.exports = listLinkedInThreads;


Read messages LinkedIn thread?



In [ ]:
var importer = require('../Core');
var moment = require('moment');
var chrono = require('chrono-node');

function getThreadParticipants(thread) {
    var profiles = [], alreadyAt = false;
    // check for thread url and go there
    return client
        .getUrl()
        .then(url => {
            return url.indexOf(thread + 'topcard/') === -1
                ? client.url('https://www.linkedin.com' + thread + 'topcard/')
                    .alertText()
                    .then(t => t.indexOf('leave') > -1 ? client.alertAccept() : '')
                    .catch(e => {
                    })
                : Promise.resolve([]);
        })
        .pause(3000)
        .elements('.view-profile')
        .then(els => Promise.all(els.value.map(a => {
            return client.elementIdAttribute(a.ELEMENT, 'href').then(h => h.value);
        })))
        .then(links => {
            return importer.runAllPromises(links.map(link => (resolve => {
                var results = [];
                return client
                    .click('a[href*="' + link.replace('https://www.linkedin.com', '') + '"]')
                    .pause(3000)
                    .then(() => readLinkedInProfileInfo(link))
                    .then(r => results = r)
                    .back()
                    .pause(1000)
                    .then(r => resolve(results))
                    .catch(e => console.log(e))
            })));
        });
};

function scrollLinkedInMessages(messages) {
    // TODO: add check for needing to go to LinkedIn
    // TODO: add check for needing to login
    // scroll to bottom of messages
    return client
        .execute(() => {
            document.getElementsByClassName('msg-s-message-list')[0]
                .scrollTop -= 10000;
        })
        .pause(2000)
        .then(() => readLinkedInMessages(messages));
}

function readLinkedInMessages(messages) {
    // TODO: check for thread id in url?
    var lastTime;
    return client
        .alertText()
        .then(t => t.indexOf('leave') > -1 ? client.alertAccept() : '')
        .catch(e => {
        })
        .then(() => getAllXPath([
            '//li[contains(@class, "msg-s-message-list__event")]',
            {
                from: './/img/@title',
                time: './/time//text()',
                message: './/*[contains(@class, "msg-s-message-listitem__message-bubble")]//text()'
            }
        ]))
        .then(r => {
            return r.map(m => {
                var newTime = chrono.parseDate(m.time + '');
                if (newTime !== null) {
                    newTime.setHours(newTime.getHours() - (new Date()).getTimezoneOffset() / 60)
                    lastTime = newTime;
                }
                return Object.assign(m, {
                    time: lastTime,
                    from: m.from + '',
                    message: m.message.filter(s => !s.match(/^\s*$/igm)).join('\n')
                })
            })
        })
        .then(r => {
            var newMessages = r
                .filter(e => messages
                    .filter(m => m.message === e.message).length === 0);
            messages = newMessages.concat(messages);
            return newMessages.length > 0
                ? scrollLinkedInMessages(messages)
                : Promise.resolve(messages);
        })
        .catch(e => console.log(e))
};

function readLinkedInThread(thread, messages) {
    var participants;
    return client
        .getUrl()
        .then(url => url.indexOf(thread) === -1
            ? client.url('https://www.linkedin.com' + thread)
                .alertText()
                .then(t => t.indexOf('leave') > -1 ? client.alertAccept() : '')
                .catch(e => {
                })
                .pause(1000)
                .isExisting('a[href*="topcard"]').then(is => is
                    ? client.click('a[href*="topcard"]')
                    : Promise.resolve([]))
            : (url.indexOf('topcard') === -1
                ? client.isExisting('a[href*="topcard"]').then(is => is
                    ? client.click('a[href*="topcard"]')
                    : Promise.resolve([]))
                : Promise.resolve([])))
        // get participants from topcard
        .pause(3000)
        .then(() => getThreadParticipants(thread))
        .then(p => participants = p)
        .back()
        // TODO: save to contacts
        .then(() => readLinkedInMessages(messages || []))
        .then(messages => ({
            thread: thread,
            participants: participants,
            messages: messages
        }))
        .catch(e => console.log(e));
};
module.exports = readLinkedInThread;


scrape linkedin threads?



In [2]:
var fs = require('fs');
var importer = require('../Core');
var glob = require('glob');
var path = require('path');
var runSeleniumCell = importer.import('selenium cell');
var loginLinkedIn, readLinkedInProfileInfo, listLinkedInThreads,
    readLinkedInThread;

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var project = PROFILE_PATH + '/Conversations';

function readThread(thread) {
    var threadId = thread.replace(/^\/|\/$/ig, '').split('/').pop();
    var file = glob.sync('**/' + threadId + '-*.json', {cwd: project})[0];
    var messages;
    try {
        messages = require(file)
    }
    catch (e) {
        messages = []
    }
    return readLinkedInThread(thread, messages)
        .then(t => {
        console.log(t);
            var filename = project + '/'
                + t.thread.replace(/^\/|\/$/ig, '').split('/').pop()
                + '-' + t.participants
                    .map(p => p.name).join('')
                    .replace(/[^a-z]/ig, '_') + '.json';
            fs.writeFileSync(filename, JSON.stringify(t, null, 4));
            return t;
        });
};

function listThreads() {
    var threads = [];
    return readLinkedInProfileInfo()
    // show my profile info
        .then(() => listLinkedInThreads(threads))
        // list all message threads
        .then(r => fs.writeFileSync(
            project + '/threads.json',
            JSON.stringify(r, null, 4)))
        .catch(e => console.log(e))
};

function scrapeLinkedInThreads() {
    var threads, fresh;

    return runSeleniumCell([
        'log in linkedin',
        'scrape LinkedIn profile',
        'list LinkedIn threads',
        'messages LinkedIn thread',
    ])
        .then(r => ({
            loginLinkedIn, readLinkedInProfileInfo, listLinkedInThreads,
            readLinkedInThread
        } = r).loginLinkedIn())
        // scrape each thread
        .then(() => {
            try {
                threads = JSON.parse(fs.readFileSync(project + '/threads.json'));
            }
            catch (e) {
                threads = [];
            }
            var threadCount = threads.length;
            var ids = threads.map(t => t.replace(/^\/|\/$/ig, '').split('/').pop());
            var files = glob.sync('**/@(' + ids.join('|') + ')-*.json', {
                cwd: project
            });
            fresh = threads.filter(t => {
                var threadId = t.replace(/^\/|\/$/ig, '').split('/').pop();
                var file = files.filter(f => f.indexOf(threadId) > -1)[0];
                return !(typeof file !== 'undefined'
                    && fs.existsSync(path.join(project, file)));
            });
            const percent = Math.round((threadCount - fresh.length) / threadCount * 100);
            console.log((threadCount - fresh.length) + ' / ' + threadCount + ' : '
                + percent
                + '%');
            console.log(fresh.slice(0, 30));
            return percent === 100 ? listThreads() : [];
        })
        .then(() => importer.runAllPromises(fresh.slice(0, 30)
            .map(t => ((resolve) => readThread(t).then((r) => resolve(r))))))
};
module.exports = scrapeLinkedInThreads;


[Function: scrapeLinkedInThreads]

In [1]:
var importer = require('../Core');
var scrapeLinkedInThreads = importer.import('scrape linkedin threads')

$$.async()
scrapeLinkedInThreads()
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e));


Initializing webdriver on localhost
[]
[19:48:48]  COMMAND	POST 	 "/wd/hub/session"
2
[19:48:50]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window_handle"
[19:48:50]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window"
[19:48:50]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/execute"
[19:48:50]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window_handles"
[19:48:50]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window"
[19:48:50]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window_handle"
[19:48:50]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window_handles"
[19:48:50]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window"
[19:48:51]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window_handles"
[19:48:51]  COMMAND	DELETE 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/window"
[19:48:51]  COMMAND	POST 	 "

[19:48:56]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/keys"
LinkedIn: Require password
[19:48:57]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element"
[19:48:57]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element/0.42132896809160925-3/click"


[19:48:57]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/keys"
[19:48:57]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element"


[19:48:57]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element/0.42132896809160925-4/submit"
[19:49:02]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/elements"
1023 / 1023 : 100%
[]
[19:49:03]  COMMAND	GET 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/url"
[19:49:03]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element"
[19:49:03]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element/0.276068254588981-1/click"
[19:49:04]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element"
[19:49:04]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element/0.276068254588981-2/click"
[19:49:06]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/elements"
[19:49:06]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element"
[19:49:06]  COMMAND	POST 	 "/wd/hub/session/65e30a37d1acb20f724dcc385caa946f/element/0.276068254588981-3/click"
Error: TypeErr

[]